In [1]:
from init import *

In [2]:
model = Model()
model.TimeStart = 0
model.TimeSteps = 1000
model.NumPaths = 110000
model.updaters.append(Updater(
    name = "IndependentBrownianMotion"
))
model.updaters.append(Updater(
    name = "SimpleGeometricalBrownianMotion",
    start = Parameter("start",130),
    args = [
        Parameter("drift",0.0),
        Parameter("diffusion",0.3)
    ]
))
model.updaters.append(Updater(name = "Error"))
fxProcessNumber = model.NumStatefulProcesses()-1
assert fxProcessNumber==0

model.updaters.append(Updater(
    name = "Product_Option",
    start = Parameter("Starting value of the option",None),
    args = [
        Parameter("myStrike",140),
        Parameter("Use 0 for call and 1 for put",1)
    ],
    refs = [fxProcessNumber]
))
callOptionOnFxProcessNumber = model.NumStatefulProcesses()-1

assert callOptionOnFxProcessNumber == (fxProcessNumber+1)

In [3]:
model.evaluations.append(EvaluationPoint(0,2))
model.json()

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 110000, "updaters": [{"name": "IndependentBrownianMotion"}, {"name": "SimpleGeometricalBrownianMotion", "start": {"name": "start", "value": 130}, "args": [{"name": "drift", "value": 0.0}, {"name": "diffusion", "value": 0.3}]}, {"name": "Error"}, {"name": "Product_Option", "start": {"name": "Starting value of the option"}, "args": [{"name": "myStrike", "value": 140}, {"name": "Use 0 for call and 1 for put", "value": 1}], "refs": [0]}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
reply = requests.post(f'{server}/model',model.json())
results = EvaluationResults(reply.json(),model)
results

2 states with 1 evaluations

In [5]:
results.df()

,name,state,time,step,npaths,mean,mean_error,stddev,skewness
0,SimpleGeometricalBrownianMotion,0,2.0,1000,110000,129.998093,0.001023,0.339187,2.133089
1,Product_Option,1,2.0,1000,4096,10.051319,0.027535,1.762223,-0.046154


# Add barrier

In [6]:
def create_single_barrier_option():
    model = Model()
    model.TimeStart = 0
    model.TimeSteps = 1000
    model.NumPaths = 100000
    model.updaters.append(Updater(
        name = "IndependentBrownianMotion"
    ))
    model.updaters.append(Updater(
        name = "SimpleGeometricalBrownianMotion",
        start = Parameter("start",130),
        args = [
            Parameter("drift",0.0),
            Parameter("diffusion",0.3)
        ]
    ))
    model.updaters.append(Updater(name = "Error"))
    fxProcessNumber = model.NumStatefulProcesses()-1
    assert fxProcessNumber==0

    model.updaters.append(Updater(
        name = "Product_Option",
        start = Parameter("Starting value of the option",None),
        args = [
            Parameter("myStrike",140),
            Parameter("Use 0 for call and 1 for put",1)
        ],
        refs = [fxProcessNumber]
    ))
    callOptionOnFxProcessNumber = model.NumStatefulProcesses()-1
    assert callOptionOnFxProcessNumber == (fxProcessNumber+1)
    
    model.updaters.append(Updater(
        name = "Barrier",
        start = Parameter("",1),
        args = [
            Parameter("trigger",131),
            Parameter("value on crossing",0),
            Parameter("direction",1),
            Parameter("set:0",0)
        ],
        refs = [fxProcessNumber]
    ))
    b1 = model.NumStatefulProcesses()-1

    
    return model

model_1b = create_single_barrier_option()
model_1b.evaluations.append(EvaluationPoint(0,2))
reply_1b = requests.post(f'{server}/model',model_1b.json())
results_1b = EvaluationResults(reply_1b.json(),model_1b)

In [7]:
results_1b.df()

,name,state,time,step,npaths,mean,mean_error,stddev,skewness
0,SimpleGeometricalBrownianMotion,0,2.0,1000,100000,129.997894,0.001129,0.357154,0.234823
1,Product_Option,1,2.0,1000,4096,10.051319,0.027535,1.762223,-0.046154
2,Barrier,2,2.0,1000,100000,0.988670,0.000335,0.105838,-9.234321
